In [248]:
from py2neo import Graph
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [249]:
graph = Graph("neo4j://localhost:7687", auth=("neo4j", neo4j_password))

In [250]:
def get_connected_nodes(node_name):
    # Define the Cypher query to find connected nodes and return both Name and Desc properties
    query = """
    MATCH (n {Name: $node_name})-[:PARENT_OF]-(connected_nodes)
    RETURN connected_nodes.Name AS name, connected_nodes.Desc AS desc
    """
    
    # Execute the query
    results = graph.run(query, node_name=node_name)
    
    # Extract the Name and Desc of each connected node
    connected_nodes_info = []
    for record in results:
        name = record["name"]
        desc = record["desc"]
        connected_nodes_info.append({"Name": name, "Desc": desc})
    
    return connected_nodes_info

In [251]:
from dotenv import load_dotenv
import os 
load_dotenv('.env')

True

In [252]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate


In [253]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="current_output", description="Output from the current agent that will be passed to the next agent. If there is no next agent, put None"),
    ResponseSchema(name="next_agent", description="The next agent to handle the request."),
    ResponseSchema(name="next_task", description="The task or action for the next agent."),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [254]:
format_instructions = output_parser.get_format_instructions()

In [255]:
search_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant designed to search and answer user's general queries
            Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents}\n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""

        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),   
        ("placeholder", "{agent_scratchpad}"),

    ]
)

greeting_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly AI that greets people warmly when they start a conversation and says goodbye appropriately when they mention leaving. 

                If someone says something about going away, respond with a proper goodbye.
                If they just greet you or talk normally, respond with a friendly greeting.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


math_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant with expertise in solving math problems. Ensure accurate calculations and provide clear explanations for mathematical concepts. Use the tavily_search_results_json tool for additional information if needed.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


tool_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant with access to powerful tools for calculations. Use these calculation tools when you need to perform precise calculations. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

supervisor_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant who is the first agent tasked to find the appropriate next agent given the task. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [256]:
import os
import openai

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')


In [257]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=OPENAI_API_KEY)

In [258]:
search = TavilySearchResults(tavily_api_key = TAVILY_API_KEY)
tools = [search]

In [259]:
from langchain.agents import create_tool_calling_agent

agent2 = create_tool_calling_agent(llm, tools, search_prompt)

In [260]:
supervisor = create_tool_calling_agent(llm, tools, supervisor_prompt)

In [261]:
agent21 = create_tool_calling_agent(llm, tools=tools, prompt= greeting_prompt)

In [262]:
# Agent 1: Generally good at math questions
agent1 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=math_prompt,
)

In [263]:
agent11 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=tool_prompt,
)

In [277]:
from langchain.agents import AgentExecutor

Supervisor_executor = AgentExecutor(agent = supervisor, tools = [search], verbose = True)

def run_query(query) :
    response = Supervisor_executor.invoke({"input" : query, "Connected_agents" : get_connected_nodes("Supervisor"), "format_instructions" : format_instructions})
    response = output_parser.parse(response['output'])
    # print("START")
    # print(response['next_agent'].lower())
    # print('END')
    while response["next_agent"] != "None" :
        query = response["next_task"]
        a = globals().get((response["next_agent"]).lower())
        agent_executor = AgentExecutor(agent=a, tools=tools, verbose=True)
        response = agent_executor.invoke({"input": query, "Connected_agents" : get_connected_nodes(str(a)), "format_instructions" : format_instructions, "current_output" : response["current_output"]})
        response=output_parser.parse(response['output'])
    return response["current_output"]


In [278]:
run_query("what is 1+1 ? ")



> Entering new AgentExecutor chain...
```json
{
	"current_output": "2",
	"next_agent": "Agent2",
	"next_task": "Perform a basic search to find interesting facts about the number 2."
}
```

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'interesting facts about the number 2'}`


[{'url': 'https://numeraly.com/about-the-number-2/', 'content': 'Fun and interesting facts about the number 2. The number 2 holds a unique position in the world of mathematics, as it is the only even prime number. This means that it is the smallest and the first even number that can only be divided by 1 and itself. Furthermore, the number 2 has a significant presence in various aspects of our lives.'}, {'url': 'https://www.thefactsite.com/number-2-facts/', 'content': "Today let's take a look at 10 fun facts about the number 2. The number two is the first prime number. There are only eight prime numbers under 20, and two is the only even number.

'Fun and interesting facts about the number 2. The number 2 holds a unique position in the world of mathematics, as it is the only even prime number. This means that it is the smallest and the first even number that can only be divided by 1 and itself. Furthermore, the number 2 has a significant presence in various aspects of our lives.'